In [1]:
%pip install pynput

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install pyautogui

  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pypro


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [66]:
import mss
import numpy as np
import cv2

def process_and_check(image_paths, screenshot):
    reference_images_data = []
    
    for i, path in enumerate(image_paths, start=1):
        ref_image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if ref_image is not None:
            label = f"Game Screen {i}"
            reference_images_data.append({"image": ref_image, "label": label})
        else:
            print(f"Error: Could not load {path}")
    
    gray_screenshot = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2GRAY)
    threshold = 0.3
    for ref in reference_images_data:
        result = cv2.matchTemplate(gray_screenshot, ref["image"], cv2.TM_SQDIFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        print(f"[DEBUG] Match results for {ref['label']}: Min Val={min_val}, Max Val={max_val}, Min Loc={min_loc}, Max Loc={max_loc}")
        
        # Use normalized min_val for thresholding
        if min_val < threshold:  # Lower values are better for TM_SQDIFF_NORMED
            print(f"In game: Matched {ref['label']}")
            return True

    print("Not in game")
    return False


image_paths = [
    "Screenshot 2024-11-30 224639.png",
    "Screenshot 2024-11-30 225724.png",
    "Screenshot 2024-11-30 224615.png"
]

In [67]:
import pyautogui
import mss
import numpy as np
import cv2
import time
import random
import pydirectinput

# Monitor region (right side of the screen)
monitor = {"top": 100, "left": 1410, "width": 1000, "height": 1800}

def move_character(direction):
    """
    Simulate key presses to move the character left or right.
    """
    if direction == "left":
        pydirectinput.press('left')  # Simulates pressing the Left Arrow key
        print("Moving Left")
        
    elif direction == "right":
        pydirectinput.press('right')  # Simulates pressing the Right Arrow key
        print("Moving Right")


def auto_play():
    with mss.mss() as sct:
        last_press_time = time.time()  # Track the time of the last press
        while True:
            # Capture the game screen
            screenshot = np.array(sct.grab(monitor))
            
            # Display the game screen in real-time
            cv2.imshow("Game Screen", cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR))
            
            # Check if 5 seconds have passed since the last key press
            current_time = time.time()
            if current_time - last_press_time >= 0.5:  # 5-second delay
                direction = random.choice(["left", "right"])
                x = process_and_check(image_paths , screenshot )
                if x :
                    move_character(direction)
                    last_press_time = current_time  # Reset the timer

            # Exit the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

        cv2.destroyAllWindows()

# Main function to run the auto-play feature
if __name__ == "__main__":
    auto_play()

[DEBUG] Match results for Game Screen 1: Min Val=0.8384070992469788, Max Val=1.0, Min Loc=(47, 1124), Max Loc=(0, 0)
[DEBUG] Match results for Game Screen 2: Min Val=0.7823712825775146, Max Val=1.0, Min Loc=(2, 1324), Max Loc=(0, 0)
[DEBUG] Match results for Game Screen 3: Min Val=1.0, Max Val=1.0, Min Loc=(0, 0), Max Loc=(0, 0)
Not in game
[DEBUG] Match results for Game Screen 1: Min Val=0.8205752968788147, Max Val=1.0, Min Loc=(45, 1124), Max Loc=(0, 0)
[DEBUG] Match results for Game Screen 2: Min Val=0.7555050253868103, Max Val=1.0, Min Loc=(4, 1324), Max Loc=(0, 0)
[DEBUG] Match results for Game Screen 3: Min Val=1.0, Max Val=1.0, Min Loc=(0, 0), Max Loc=(0, 0)
Not in game
[DEBUG] Match results for Game Screen 1: Min Val=0.8185468316078186, Max Val=1.0, Min Loc=(45, 1124), Max Loc=(0, 0)
[DEBUG] Match results for Game Screen 2: Min Val=0.7528480887413025, Max Val=1.0, Min Loc=(7, 1324), Max Loc=(0, 0)
[DEBUG] Match results for Game Screen 3: Min Val=1.0, Max Val=1.0, Min Loc=(0, 0)

## **Keep this code for debugging !**

In [61]:
import cv2
import numpy as np

def process_and_check(image_paths, screenshot_path):
    """
    Processes the screenshot and checks for matches with reference images.
    Draws rectangles around matched templates for debugging.

    Parameters:
        image_paths (list): Paths to the reference images.
        screenshot_path (str): Path to the saved game screenshot.

    Returns:
        bool: True if any reference image matches the screenshot, otherwise False.
    """
    reference_images_data = []

    print("Loading reference images...")
    # Load reference images
    for i, path in enumerate(image_paths, start=1):
        ref_image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if ref_image is not None:
            label = f"Game Screen {i}"
            reference_images_data.append({"image": ref_image, "label": label})
            print(f"[INFO] Loaded reference image: {label} from {path}")
        else:
            print(f"[ERROR] Failed to load {path}")

    # Load the saved screenshot as grayscale
    print(f"Loading screenshot from: {screenshot_path}")
    screenshot = cv2.imread(screenshot_path, cv2.IMREAD_GRAYSCALE)
    colored_screenshot = cv2.imread(screenshot_path)  # For drawing rectangles
    if screenshot is None:
        print(f"[ERROR] Failed to load screenshot: {screenshot_path}")
        return False

    # Perform template matching
    print("Starting template matching...")
    matched = False
    threshold = 0.8  # Adjust this threshold for normalized results
    for ref in reference_images_data:
        print(f"[DEBUG] Matching with template: {ref['label']}")

        # Perform template matching using normalized squared differences
        result = cv2.matchTemplate(screenshot, ref["image"], cv2.TM_SQDIFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

        print(f"[DEBUG] Match results for {ref['label']}: Min Val={min_val}, Max Val={max_val}, Min Loc={min_loc}, Max Loc={max_loc}")

        # Use normalized min_val for thresholding
        if min_val < threshold:  # For TM_SQDIFF_NORMED, lower values indicate better matches
            matched = True
            print(f"[SUCCESS] In game: Matched {ref['label']} with score {min_val}")

            # Draw a rectangle around the matched area
            top_left = min_loc
            h, w = ref["image"].shape
            bottom_right = (top_left[0] + w, top_left[1] + h)
            cv2.rectangle(colored_screenshot, top_left, bottom_right, (0, 255, 0), 2)  # Green rectangle
            print(f"[DEBUG] Rectangle drawn at: Top Left={top_left}, Bottom Right={bottom_right}")

    # Save the screenshot with rectangles for debugging
    debug_output_path = "debug_matched_screenshot_sqdiff.png"
    cv2.imwrite(debug_output_path, colored_screenshot)
    print(f"[DEBUG] Saved matched screenshot with rectangles as '{debug_output_path}'")

    # Display the result for debugging
    cv2.imshow("Matched Screenshot", colored_screenshot)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    if not matched:
        print("[INFO] Not in game: No templates matched.")
    return matched


# Paths to reference images
image_paths = [
    "Screenshot 2024-11-30 224639.png",
    "Screenshot 2024-11-30 225724.png",
    "Screenshot 2024-11-30 224615.png"
]

# Path to the saved screenshot
screenshot_path = "C://Users//shaym//Downloads//game_screen_5.png"  # Replace with your saved screenshot file path

# Run the function
process_and_check(image_paths, screenshot_path)

Loading reference images...
[INFO] Loaded reference image: Game Screen 1 from Screenshot 2024-11-30 224639.png
[INFO] Loaded reference image: Game Screen 2 from Screenshot 2024-11-30 225724.png
[INFO] Loaded reference image: Game Screen 3 from Screenshot 2024-11-30 224615.png
Loading screenshot from: C://Users//shaym//Downloads//game_screen_5.png
Starting template matching...
[DEBUG] Matching with template: Game Screen 1
[DEBUG] Match results for Game Screen 1: Min Val=1.0, Max Val=1.0, Min Loc=(0, 0), Max Loc=(0, 0)
[DEBUG] Matching with template: Game Screen 2
[DEBUG] Match results for Game Screen 2: Min Val=0.10586526244878769, Max Val=0.7621999979019165, Min Loc=(70, 1112), Max Loc=(327, 847)
[SUCCESS] In game: Matched Game Screen 2 with score 0.10586526244878769
[DEBUG] Rectangle drawn at: Top Left=(70, 1112), Bottom Right=(768, 1288)
[DEBUG] Matching with template: Game Screen 3
[DEBUG] Match results for Game Screen 3: Min Val=0.5480519533157349, Max Val=0.9911457300186157, Min L

True